# Очистка данных

## Чтение данных

Загрузим данные на машину (colab only)

In [1]:
# from google.colab import drive
# drive.mount('/gdrive')
# !cp -r /gdrive/Othercomputers/"My Zenbook"/xgbse_analytics/lending_data ./lending_data

Прочитаем данные о одобренных кредитах

In [2]:
import pandas as pd
pd.set_option('display.max_rows', None)

In [3]:
accepted = pd.read_csv('./lending_data/accepted.csv', low_memory=False)
accepted.head().transpose()

,0,1,2,3,4
id,68407277,68355089,68341763,66310712,68476807
member_id,NaN,NaN,NaN,NaN,NaN
loan_amnt,3600.0,24700.0,20000.0,35000.0,10400.0
funded_amnt,3600.0,24700.0,20000.0,35000.0,10400.0
funded_amnt_inv,3600.0,24700.0,20000.0,35000.0,10400.0
term,36 months,36 months,60 months,60 months,60 months
int_rate,13.99,11.99,10.78,14.85,22.45
installment,123.03,820.28,432.66,829.9,289.91
grade,C,C,B,C,F
sub_grade,C4,C1,B4,C5,F1


In [4]:
origin_cols = set(accepted.columns)

In [5]:
accepted.shape

(2260701, 151)

## Обработаем колонки

### Удалим колонки с уникальными идентификаторами

- `id` - уникальный идентификатор кредита, для каждого платежа новый.
- `member_id` - уникальный идентификатор кредитора.
- `url` - Уникальная ссылка на страницу займа.

In [6]:
unique_id_cols = [
    'id',
    'member_id',
    'url',
]
accepted.drop(columns=unique_id_cols, inplace=True)

### Выбросим все колонки, дублирующие данные


- `grade` - Отражен в `sub_grade`
- `int_rate` - Отражен в `sub_grade`
- `zip_code` - Зашифрован до первых 3 цифр, отражает то же самое, что и `addr_state`, но менее интерпретируем.

In [7]:
redundand_cols = [
    'grade',
    'int_rate',
    'zip_code',
]
accepted.drop(columns=redundand_cols, inplace=True)

### Выбросим колонки, которые сложно обрабатывать

- `desc` - текстовое описание цели кредита.
- `title` - Заголовок цели кредита. Частично отражен в `purpose`.
- `emp_title` - название должности.

In [8]:
accepted.drop(columns=['desc', 'title', 'emp_title'], inplace=True)

### Переведем колонки с датами в тип datetime

In [9]:
accepted['issue_d'] = pd.to_datetime(accepted['issue_d'], format='%b-%Y')

accepted['earliest_cr_line'] = pd.to_datetime(accepted['earliest_cr_line'],
                                              format='%b-%Y')
accepted['last_credit_pull_d'] = pd.to_datetime(accepted['last_credit_pull_d'],
                                                format='%b-%Y')

accepted['last_pymnt_d'] = pd.to_datetime(accepted['last_pymnt_d'],
                                          format='%b-%Y')

### Удалим все колонки, приводящие к утечке целевой переменной



- `funded_amnt` - Отражает информацию о выплаченной сумме у завершенных кредитов.
- `funded_amnt_inv` - Отражает информацию о выплаченной сумме у завершенных кредитов, для части от инвесторов.
- `out_prncp` - Отражает информацию о оставшейся к выплате суммы.
- `out_prncp_inv`- Отражает информацию о оставшейся к выплате суммы, для части от инвесторов.
- `total_pymnt` - Обьем уже полученных платежей.
- `total_pymnt_inv` - Обьем уже полученных платежей, для части от инвесторов.
- `total_rec_prncp` - Сумма уже выплаченного основного долга
- `total_rec_int` - Обьем выплаченных процентов по кредиту.
- `total_rec_late_fee` - Обьем пени за просрочку по кредиту.
- `recoveries` - Обьем возвращенных средств после дефолта, > 0 означает что дефолт произошел.  
- `collection_recovery_fee` - Обьем возвращенных средств после списания долга. Не равно нулю только в случае, если дефолт по долгу произошел
- `last_pymnt_d` -  Месяц последнего платежа(Пока не удаляем, необходим для целевой переменной)
- `last_pymnt_amnt` - Сумма выплат на момент последнего платежа.
- `debt_settlement_flag` - Флаг, что должник работает с коллекторской компанией
- `debt_settlement_flag_date` - Дата установки флага `debt_settlement_flag`
- `settlement_status` - Статус дефолтного плана выплат.
- `settlement_date` - Дата заключения дефолтного плана выплат.
- `settlement_amount` - Сумма дефолтного плана выплат.
- `settlement_percentage` - Сумма дефолтного плана как процент от выданных средств.
- `settlement_term` - Срок соглашения дефолтных выплат
- `pymnt_plan` - был ли заключен новый план выплат(заключается после дефолта)
- `revol_bal` - оборотный остаток по кредиту.
- `revol_util` - утилизация оборотного остатка.
- `hardship_flag` - Обращался ли клиент за перерасчетом долга.
- `hardship_type` - Тип перерасчета
- `hardship_reason` - Причина перерасчета
- `hardship_status` - Статус перерасчета
- `deferral_term` - Количество месяцев уменьшенных платежей
- `hardship_amount` - Сумма выплат 
- `hardship_start_date` - Дата старта по перерасчету
- `hardship_end_date` - Окончание периода перерасчета
- `payment_plan_start_date` - Дата старта плана выплат
- `hardship_length` - Период перерасчета
- `hardship_dpd` - Остаток периода перерасчета
- `hardship_loan_status` - Статус кредита на момент перерасчета
- `orig_projected_additional_accrued_interest` - Процент, полученный с плана перерасчета
- `hardship_payoff_balance_amount` - Сумма выплат на начало плана перерасчета
- `hardship_last_payment_amount` - Сумма выплат на конец плана перерасчета
- `last_fico_range_high` - наивысший кредитный рейтинг на время последнего запроса
- `last_fico_range_low` - наихудший кредитный рейтинг на время последнего запроса

In [10]:
leak_cols = [
    'funded_amnt',
    'funded_amnt_inv',
    'out_prncp',
    'out_prncp_inv',
    'total_pymnt',
    'total_pymnt_inv',
    'total_rec_prncp',
    'total_rec_int',
    'total_rec_late_fee',
    'recoveries',
    'collection_recovery_fee',
    # 'last_pymnt_d', # Leaks
    'last_pymnt_amnt',
    'debt_settlement_flag',
    'debt_settlement_flag_date',
    'settlement_status',
    'settlement_date',
    'settlement_amount',
    'settlement_percentage',
    'settlement_term',
    'pymnt_plan',
    'revol_bal',
    'revol_util',
    'hardship_flag',
    'hardship_type',
    'hardship_reason',
    'hardship_status',
    'deferral_term',
    'hardship_amount',
    'hardship_start_date',
    'hardship_end_date',
    'payment_plan_start_date',
    'hardship_length',
    'hardship_dpd',
    'hardship_loan_status',
    'orig_projected_additional_accrued_interest',
    'hardship_payoff_balance_amount',
    'hardship_last_payment_amount',
    'last_fico_range_high',
    'last_fico_range_low'
]

accepted.drop(columns=leak_cols, inplace=True)
accepted.head().transpose()

,0,1,2,3,4
loan_amnt,3600.0,24700.0,20000.0,35000.0,10400.0
term,36 months,36 months,60 months,60 months,60 months
installment,123.03,820.28,432.66,829.9,289.91
sub_grade,C4,C1,B4,C5,F1
emp_length,10+ years,10+ years,10+ years,10+ years,3 years
home_ownership,MORTGAGE,MORTGAGE,MORTGAGE,MORTGAGE,MORTGAGE
annual_inc,55000.0,65000.0,63000.0,110000.0,104433.0
verification_status,Not Verified,Not Verified,Not Verified,Source Verified,Source Verified
issue_d,2015-12-01 00:00:00,2015-12-01 00:00:00,2015-12-01 00:00:00,2015-12-01 00:00:00,2015-12-01 00:00:00
loan_status,Fully Paid,Fully Paid,Fully Paid,Current,Fully Paid


## Обработаем строки

### Обработаем статус займа.

Нас интересуют завершенные займы, поэтому удалим займы в следующих статусах:

- Default
- Current
- Late (31-120 days)
- Late (16-30 days)
- In Grace Period

Также уберем записи со статусами:

- Does not meet the credit policy. Status:Fully Paid
- Does not meet the credit policy. Status:Charged Off

Эти кредиты были одобрены ранее, но не проходят для существующей кредитной политики, и скорее всего будут иметь другое распределение, что будет иметь негативный эффект на результаты нашего исследования.

In [11]:
loan_stat_delete = ['Default',
                    'Does not meet the credit policy. Status:Charged Off',
                    'Does not meet the credit policy. Status:Fully Paid',
                    'In Grace Period',
                    'Late (16-30 days)',
                    'Late (31-120 days)',
                    'Current']

accepted = accepted[~accepted['loan_status'].isin(loan_stat_delete)]
accepted['loan_status'].value_counts(dropna=False)

loan_status
Fully Paid     1076751
Charged Off     268559
NaN                 33
Name: count, dtype: int64

### Обработаем тип заявки

Будем брать только индивидуальные заявки.

In [12]:
accepted = accepted[accepted['application_type'] == 'Individual']
accepted['application_type'].unique()

array(['Individual'], dtype=object)

Удалим сопутствующие колонки

In [13]:
sec_app_cols = ["application_type",
                "annual_inc_joint",
                "dti_joint",
                "verification_status_joint",
                "revol_bal_joint",
                "sec_app_fico_range_low",
                "sec_app_fico_range_high",
                "sec_app_earliest_cr_line",
                "sec_app_inq_last_6mths",
                "sec_app_mort_acc",
                "sec_app_open_acc",
                "sec_app_revol_util",
                "sec_app_open_act_il",
                "sec_app_num_rev_accts",
                "sec_app_chargeoff_within_12_mths",
                "sec_app_collections_12_mths_ex_med",
                "sec_app_mths_since_last_major_derog"]

accepted.drop(columns=sec_app_cols, inplace=True)

### Посмотрим на пропуски

В нашем случае существуют 2 типа пропусков:
- Пропуски, которые были проставленны намеренно, например, в колонке `mths_since_last_delinq` пропуск означает, что кредитуемый не допускал нарушений в принципе.
- Пропуски, которые были допущенны в силу неполноты данных.

In [14]:
nan_content = dict()

for i, col in enumerate(accepted.columns):
    vals = [accepted[col].isna().sum(), len(accepted[col])]
    labels = ['nan', 'Not nan']
    if vals[0]/vals[1] != 0:
        nan_content[col] = vals[0]/vals[1]

In [15]:
nan_order = dict(sorted(nan_content.items(), key=lambda item: item[1]))

labels = list(nan_order.keys())
vals = list(nan_order.values())


nan_order

{'inq_last_6mths': 7.578570833112292e-07,
 'tax_liens': 2.9556426249137938e-05,
 'last_credit_pull_d': 4.0924282498806374e-05,
 'collections_12_mths_ex_med': 4.2439996665428835e-05,
 'chargeoff_within_12_mths': 4.2439996665428835e-05,
 'pub_rec_bankruptcies': 0.0005282263870679268,
 'last_pymnt_d': 0.001660464869534903,
 'acc_open_past_24mths': 0.035832240756038226,
 'mort_acc': 0.035832240756038226,
 'total_bal_ex_mort': 0.035832240756038226,
 'total_bc_limit': 0.035832240756038226,
 'num_bc_sats': 0.04231949738918235,
 'num_sats': 0.04231949738918235,
 'mths_since_recent_bc': 0.045193291449098526,
 'bc_open_to_buy': 0.045876120681161946,
 'percent_bc_gt_75': 0.04618835779948617,
 'bc_util': 0.0464467870648953,
 'tot_coll_amt': 0.051175815264757375,
 'tot_cur_bal': 0.051175815264757375,
 'total_rev_hi_lim': 0.051175815264757375,
 'mo_sin_rcnt_tl': 0.051175815264757375,
 'num_accts_ever_120_pd': 0.051175815264757375,
 'num_actv_bc_tl': 0.051175815264757375,
 'num_actv_rev_tl': 0.051175

### Удалим пропуски в колонках

Будем удалять записи в колонках, в которых пропусков не более 15%.
В таком случае целостность данных не пострадает.

In [16]:
just_drop_cols = ['inq_last_6mths',
                  'tax_liens',
                  'last_credit_pull_d',
                  'collections_12_mths_ex_med',
                  'chargeoff_within_12_mths',
                  'pub_rec_bankruptcies',
                  'acc_open_past_24mths',
                  'mort_acc',
                  'total_bal_ex_mort',
                  'total_bc_limit',
                  'num_bc_sats',
                  'num_sats',
                  'mths_since_recent_bc',
                  'bc_open_to_buy',
                  'percent_bc_gt_75',
                  'bc_util',
                  'tot_coll_amt',
                  'tot_cur_bal',
                  'total_rev_hi_lim',
                  'mo_sin_rcnt_tl',
                  'num_accts_ever_120_pd',
                  'num_actv_bc_tl',
                  'num_actv_rev_tl',
                  'num_bc_tl',
                  'num_il_tl',
                  'num_op_rev_tl',
                  'num_rev_tl_bal_gt_0',
                  'num_tl_30dpd',
                  'num_tl_90g_dpd_24m',
                  'num_tl_op_past_12m',
                  'tot_hi_cred_lim',
                  'total_il_high_credit_limit',
                  'mo_sin_old_rev_tl_op',
                  'mo_sin_rcnt_rev_tl_op',
                  'num_rev_accts',
                  'avg_cur_bal',
                  'pct_tl_nvr_dlq',
                  'emp_length',
                  'mo_sin_old_il_acct',
                  'num_tl_120dpd_2m',
                  'mths_since_recent_inq',
                  'last_pymnt_d']

accepted.dropna(subset=just_drop_cols, inplace=True)

Для переменных формата `mths_since_***` отсутствие значения показывает, что события не произошло.
Поэтому для таких переменных мы будем заводить еще одну колонку-флаг, произошло ли событие вообще, а значения заполнять очень большим числом

In [17]:
time_since = [
    'mths_since_last_delinq',
    'mths_since_recent_revol_delinq',
    'mths_since_last_major_derog',
    'mths_since_recent_bc_dlq',
    'mths_since_last_record',
]

for col_name in time_since:
    max_val = accepted[col_name].max()
    fill_val = max_val**2 + 1000  # Fill with unrealistic value

    new_col_name = f'has_been_{col_name.lstrip("mths_since_")}'

    accepted[new_col_name] = accepted[col_name].isna().astype('int')
    accepted[col_name].fillna(fill_val, inplace=True)

Заметим, что для части колонок отсутствуют целые блоки данных. Удалим колонки, в которых пропусков больше 50%. Данных без них все еще достаточно, чтобы сделать выводы.

In [18]:
missing_cols = [i for i in nan_order if nan_order[i] >= 0.5]


accepted.drop(columns=missing_cols, inplace=True)

### Обработаем время жизни клиента

In [19]:
accepted['term'].value_counts(dropna=False)

term
 36 months    734805
 60 months    248293
Name: count, dtype: int64

В колонке `term` ровно 2 значения: 36 или 60 месяцев, поэтому просто отобразим значения в числа.

In [20]:
accepted['term'] = accepted['term'].map({' 36 months': 36, ' 60 months': 60})

### Выделим целевую переменную

В рамках задачи, целевая переменная состоит из двух колонок: `duration` и `event`.
Первая отвечает за время наблюдения за клиентом, вторая показывает, произошло ли событие(дефолт по кредиту) за время наблюдений.

In [21]:
def extract_target(row):
    if row['loan_status'] != 'Fully Paid':
        return int((row['last_pymnt_d'] - row['issue_d']).components.days/30)

    return row['term']


accepted['duration'] = accepted.apply(extract_target, axis=1)
accepted['event'] = (accepted['loan_status'] != 'Fully Paid')
accepted.drop(columns=['last_pymnt_d', 'loan_status', 'term'], inplace=True)
accepted[['duration', 'event']].head(15)

,duration,event
0,36,False
1,36,False
4,60,False
6,36,False
7,36,False
8,36,False
9,36,False
12,36,False
13,17,True
15,36,False


### Подготовка категорий

В данных встречаются колонки `emp_length` и `sub_grade`.
Эти колонки являются категориальными, но при этом содержат в себе порядок.
Будем кодировать эти колонки вручную, согласно внутреннему порядку.

In [22]:
mapping_dict = {
                "emp_length": {
                    "10+ years": 10,
                    "9 years": 9,
                    "8 years": 8,
                    "7 years": 7,
                    "6 years": 6,
                    "5 years": 5,
                    "4 years": 4,
                    "3 years": 3,
                    "2 years": 2,
                    "1 year": 1,
                    "< 1 year": 0,
                    "n/a": 0
                },
                "sub_grade":
                    dict([(val, i+1) for i, val in enumerate(sorted(accepted['sub_grade'].unique()))])
                }

accepted = accepted.replace(mapping_dict)
accepted[['emp_length', 'sub_grade']].head()

,emp_length,sub_grade
0,10,14
1,10,11
4,3,26
6,10,7
7,10,6


### Удалим колонки с единственным значением.

После всех действий у нас остались "мусорные колонки", содержащие одно значение. Удалим их.

In [23]:
accepted = accepted.loc[:,accepted.apply(pd.Series.nunique) != 1]

## Выгрузка данных

In [24]:
accepted.isna().sum()

loan_amnt                       0
installment                     0
sub_grade                       0
emp_length                      0
home_ownership                  0
annual_inc                      0
verification_status             0
issue_d                         0
purpose                         0
addr_state                      0
dti                             0
delinq_2yrs                     0
earliest_cr_line                0
fico_range_low                  0
fico_range_high                 0
inq_last_6mths                  0
open_acc                        0
pub_rec                         0
total_acc                       0
initial_list_status             0
last_credit_pull_d              0
collections_12_mths_ex_med      0
acc_now_delinq                  0
tot_coll_amt                    0
tot_cur_bal                     0
total_rev_hi_lim                0
acc_open_past_24mths            0
avg_cur_bal                     0
bc_open_to_buy                  0
bc_util       

In [25]:
accepted.shape

(983098, 69)

In [26]:
accepted.head().transpose()

,0,1,4,6,7
loan_amnt,3600.0,24700.0,10400.0,20000.0,20000.0
installment,123.03,820.28,289.91,637.58,631.26
sub_grade,14,11,26,7,6
emp_length,10,10,3,10,10
home_ownership,MORTGAGE,MORTGAGE,MORTGAGE,MORTGAGE,MORTGAGE
annual_inc,55000.0,65000.0,104433.0,180000.0,85000.0
verification_status,Not Verified,Not Verified,Source Verified,Not Verified,Not Verified
issue_d,2015-12-01 00:00:00,2015-12-01 00:00:00,2015-12-01 00:00:00,2015-12-01 00:00:00,2015-12-01 00:00:00
purpose,debt_consolidation,small_business,major_purchase,debt_consolidation,major_purchase
addr_state,PA,SD,PA,MN,SC


In [27]:
accepted.to_csv('lending_data/accepted_clean.csv', index=False)

Colab only

In [28]:
# !cp ./lending_data/accepted_clean.csv /gdrive/Othercomputers/"My Zenbook"/xgbse_analytics/lending_data/accepted_clean.csv